In [ ]:
import os
import random
import numpy as np
import warnings

In [ ]:
import energy_model
import utils

In [ ]:
import importlib
importlib.reload(energy_model)
importlib.reload(utils)

In [ ]:
from energy_model import EnergyModel
from utils.data_handling import ScenarioData
from utils.gurobi_env import get_Gurobi_WLS_env

In [ ]:
from configs.config import *

In [ ]:
# create test scenario data

ts_dict = {
    'load_level': 2.5e5,
    'wind_year': 2010,
    'solar_year': 2010,
    'price_year': 2023,
    'carbon_year': 2023
}

cost_dict = { # sense check costs by LCOE
    'wind_capex': 5e3, # €/kWp
    'wind_opex': 100, # €/kWp/yr
    'wind_lifetime': 20, # years
    'solar_capex': 1e3, # €/kWp
    'solar_opex': 10, # €/kWp/yr
    'solar_lifetime': 20, # years
    'carbon_price': 1.0 # €/kgCO2
}

storage_dict = {
    'li-ion': {
        'cost': 300,
        'lifetime': 12,
        'efficiency': 0.9,
        'discharge_ratio': 0.4
    },
    'other': {
        'cost': 100,
        'lifetime': 20,
        'efficiency': 0.5,
        'discharge_ratio': 0.2
    },
    'other2': {
        'cost': 175,
        'lifetime': 20,
        'efficiency': 0.8,
        'discharge_ratio': 0.2
    }
}

In [ ]:
scenarios = []
random.seed(42)
for m in range(1):
    ts_dict['wind_year'] = random.choice(wind_years)
    ts_dict['solar_year'] = random.choice(solar_years)
    scenario = ScenarioData(ts_dict, cost_dict, storage_dict)
    scenario.load_timeseries(dataset_dir)
    scenarios.append(scenario)

In [ ]:
settings = {
    'T':8760,
    'initial_SoC':0.5,
    'max_storage_cap':1e9, # slack bound
    'N_technologies': 2,
    'allow_elec_purchase': True,
    'grid_capacity': 500e3, # kW
    'capex_budget': 10e9/20, # €/yr
    'solar_capacity_limit': 500e3, # kWp
    #'technologies_to_use': ['li-ion'],
}

Note:
1. You need to be careful that the capex budget is sufficiently large to allow the purchase of enough generation capacity to satisfy the plant demand
2. The solar capacity limit is needed to make sure that the optimised solar capacity is not infeasibly large due to land area, solar PV is approx 1kWp/m2

In [ ]:
model = EnergyModel()
with warnings.catch_warnings():
    # filter pandas warnings, `DeprecationWarning: np.find_common_type is deprecated.`
    warnings.simplefilter("ignore", category=FutureWarning)
    model.generate_SP(scenarios,settings)

In [ ]:
env = get_Gurobi_WLS_env(silence=False)
model.solve(env=env)

In [ ]:
print(model.model.objective)
print(model.scenarios[0].load @ model.scenarios[0].elec_prices)
print(model.corrected_objective)

In [ ]:
print('wind capacity', model.model.variables.wind_capacity.solution.values)
print('solar capacity', model.model.variables.solar_capacity.solution.values)
print('storage capacities', [getattr(model.model.variables,f'{tech}_capacity').solution.values for tech in model.techs])

In [ ]:
for key,val in model.scen_obj_contrs[0].items():
    print(key,val.solution.values)

In [ ]:
print(sum([model.scen_obj_contrs[0][key] for key in ['wind','solar','storage']]).solution.values)

In [ ]:
energy_flares = model.get_flared_energy()
print(energy_flares['s0']['energy_dump'].sum().values)
print(energy_flares['s0']['generation_curtailment'].sum().values)
# ToDo plot energy flares against generation & electricity price

In [ ]:
battery_cycles = model.get_battery_cycles()
print(battery_cycles)

In [ ]:
from utils.plotting import init_profile_fig, add_profile

In [ ]:
fig = init_profile_fig(y_titles={'y1':'Energy flow (kWh)', 'y2':'State of Charge (kWh)', 'y3':'Price ($/kWh)'})

fig = add_profile(fig, model.grid_energies[0].solution.values, name='Grid load')
fig = add_profile(fig, model.scenarios[0].norm_wind_gen*model.model.variables.wind_capacity.solution.values, name=f'Wind generation')
fig = add_profile(fig, model.scenarios[0].norm_solar_gen*model.model.variables.solar_capacity.solution.values, name=f'Solar generation')
fig = add_profile(fig, model.scenarios[0].elec_prices, name='Electricity price', yaxis='y3', visible='legendonly')
for i,tech in enumerate(model.techs):
    if model.model.variables[f'{tech}_capacity'].solution > 0:
        fig = add_profile(fig, getattr(model.model.variables,f'SOC_i{i}_s0').solution, name=f'{tech} SoC', yaxis=f'y2', visible='legendonly')

#fig['layout']['xaxis'].update(range=['2000-04-24','2000-05-01'])
fig.write_html(f'temp_plot.html')
fig.show()

In [ ]:
fig = init_profile_fig(y_titles={'y1':'Energy flow (kWh)', 'y2':'State of Charge (kWh)', 'y3':'Price ($/kWh)'})

fig = add_profile(fig, model.grid_energies[0].solution.values, name='Grid load')
fig = add_profile(fig, energy_flares['s0']['energy_dump'].values, name=f'Energy dumped')
fig = add_profile(fig, energy_flares['s0']['generation_curtailment'].values, name=f'Generation curtailment')
fig = add_profile(fig, model.scenarios[0].norm_wind_gen*model.model.variables.wind_capacity.solution.values + model.scenarios[0].norm_solar_gen*model.model.variables.solar_capacity.solution.values, name=f'Total generation')
fig = add_profile(fig, model.scenarios[0].elec_prices, name='Electricity price', yaxis='y3', visible='legendonly')

#fig['layout']['xaxis'].update(range=['2000-04-24','2000-05-01'])
fig.write_html(f'temp_plot2.html')
fig.show()